## 1️⃣ Cài đặt môi trường

## 🧩 Yêu cầu phụ thuộc (Dependency) cho Google Colab

Để chạy dự án ComparisonDetector ổn định trên Google Colab, vui lòng đảm bảo các yêu cầu sau:

- TensorFlow: `tensorflow==2.19.0` (chạy ở chế độ `tf.compat.v1`, tắt eager)
- Slim: `tf_slim==1.1.0` (thay thế `tf.contrib.slim`)
- NumPy: `numpy==1.26.4` (tránh xung đột ABI với OpenCV và TF)
- OpenCV: `opencv-python-headless==4.7.0.72` (không dùng bản có GUI)
- Khoa học dữ liệu: `scikit-image==0.21.0`, `scipy==1.11.4`, `matplotlib==3.7.3`, `tqdm==4.66.4`

Lưu ý quan trọng:

- Không cài `tensorflow-io-gcs-filesystem` (không tương thích với Python 3.12 trên Colab).
- Gỡ các gói có sẵn trên Colab nếu gây xung đột: `jax`, `jaxlib`, `albumentations`, `albucore`, `pytensor`, `shap`, `opencv-python`, `opencv-contrib-python`.
- Ưu tiên dùng bánh xe (wheels) bằng cờ `--prefer-binary` để tránh build từ source.

Sau đó, cài đặt theo Cell 2 để thiết lập môi trường phù hợp.

In [ ]:
# Cài đặt dependencies ổn định cho Colab (tối ưu tốc độ và tránh xung đột)
# 1) Gỡ các gói không cần thiết hoặc gây xung đột với phiên bản pin của dự án
!pip uninstall -y -q opencv-python opencv-contrib-python opencv-python-headless jax jaxlib shap albumentations albucore pytensor || true

# 2) Cài đặt stack tương thích với TensorFlow 2.19 (không cài tensorflow-io-gcs-filesystem)
#    Dùng --prefer-binary để ưu tiên wheels (nhanh hơn, không build từ source)
PIP_FLAGS = "--prefer-binary"
!pip install -q {PIP_FLAGS} tensorflow==2.19.0 tf_slim==1.1.0
!pip install -q {PIP_FLAGS} numpy==1.26.4 opencv-python-headless==4.7.0.72
!pip install -q {PIP_FLAGS} scikit-image==0.21.0 scipy==1.11.4 matplotlib==3.7.3 tqdm==4.66.4

# 3) In xác nhận
print("✓ Đã cài đặt dependencies tương thích và gỡ xung đột")

In [ ]:
# Kiểm tra các gói gây xung đột đã được gỡ bỏ và phiên bản hiện tại
import pkgutil, sys

def is_installed(pkg):
    return pkgutil.find_loader(pkg) is not None

check_list = [
    ("numpy", "1.26.4"),
    ("cv2", "4.7.0.72"),
]

print("\n=== Version check ===")
try:
    import numpy as np
    print("numpy:", np.__version__)
except Exception as e:
    print("numpy not found:", e)

try:
    import cv2
    print("opencv-python-headless (cv2):", cv2.__version__)
except Exception as e:
    print("cv2 not found:", e)

# Cảnh báo các gói không cần thiết nếu vẫn còn trong môi trường
maybe_conflicts = ["jax", "jaxlib", "albumentations", "albucore", "pytensor", "shap", "opencv_python", "opencv_contrib_python"]
leftovers = [p for p in maybe_conflicts if is_installed(p.replace('-', '_'))]
if leftovers:
    print("\nCảnh báo: các gói sau vẫn tồn tại và có thể gây cảnh báo phụ thuộc:")
    for p in leftovers:
        print("-", p)
    print("Nếu không dùng chúng, hãy gỡ bằng: !pip uninstall -y", " ".join(leftovers))
else:
    print("\nKhông còn gói xung đột đáng kể.")

In [ ]:
# Kiểm tra phiên bản và disable eager execution
import tensorflow as tf
import numpy as np
tf.compat.v1.disable_eager_execution()

print(f"TensorFlow: {tf.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Eager execution: {tf.executing_eagerly()}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 2️⃣ Clone repo và setup

In [ ]:
# Clone repo từ GitHub
!git clone https://github.com/dungdinhhaha/AIDetect.git
%cd AIDetect
!ls -la

In [ ]:
# Mount Google Drive (để lưu model và data)
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục cho data (nếu chưa có)
!mkdir -p /content/data/tct
print("✓ Google Drive mounted and data folder created")

## 3️⃣ Chuẩn bị dữ liệu

**Tùy chọn A**: Upload TFRecord từ Drive
```python
!cp /content/drive/MyDrive/your-data/*.tfrecord /content/data/tct/
!cp /content/drive/MyDrive/your-data/labels.tsv /content/data/tct/
```

**Tùy chọn B**: Sử dụng data mẫu (nếu có trong repo)
```python
# Uncomment nếu repo có sample data
# !cp -r sample_data/* /content/data/tct/
```

In [ ]:
# THAY ĐỔI đường dẫn này tùy theo vị trí data của bạn trên Drive
# Ví dụ: nếu data ở MyDrive/ComparisonDetector_Data/

# !cp /content/drive/MyDrive/ComparisonDetector_Data/*.tfrecord /content/data/tct/
# !cp /content/drive/MyDrive/ComparisonDetector_Data/labels.tsv /content/data/tct/

# Kiểm tra data đã có chưa
!ls -lh /content/data/tct/

## 4️⃣ Cấu hình training

In [ ]:
# Cập nhật config để trỏ đến data path Colab
config_content = '''
import os

class Config:
    # Paths
    DATA_DIR = "/content/data/tct"  # Colab data path
    MODLE_DIR = "/content/drive/MyDrive/comparison_detector_models"  # Lưu model vào Drive
    CHECKPOINT_DIR = None  # Set path nếu có pretrained checkpoint
    
    # Network
    NET_NAME = "resnet_v1_101"  # hoặc resnet_v1_50
    NUM_CLASS = 12  # 11 classes + 1 background
    NUM_SUPPROTS = 3  # Số ảnh reference mỗi class
    ROI_SIZE = 7
    
    # Training
    BATCH_SIZE = 1  # Colab GPU nhỏ nên để 1
    EPOCH = 50
    LEARNING_RATE = 0.001
    MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0001
    BOUNDARY = [30000, 50000]
    
    # Save
    SAVE_EVERY_N_STEP = 500
    
    # GPU
    GPU_GROUPS = [["GPU:0"]]  # Colab single GPU
    
    # Image preprocessing
    PIXEL_MEANS = [123.68, 116.779, 103.939]  # ImageNet mean
'''

with open('configs/config.py', 'w') as f:
    f.write(config_content)

print("✓ Config updated for Colab")
!cat configs/config.py

## 5️⃣ Training

In [ ]:
# Chạy training (sẽ mất vài giờ tùy dataset)
!python tools/train.py

## 6️⃣ Theo dõi training với TensorBoard (optional)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/comparison_detector_models

## 7️⃣ Smoke test (kiểm tra code không cần train)

In [ ]:
# Test nhanh import và graph build (không cần data thật)
!python tools/smoke_test.py

---

## 📝 Lưu ý quan trọng

### ✅ Trước khi train
1. **GPU Runtime**: Đảm bảo đã chọn GPU runtime (T4 trở lên)
2. **Data**: Upload TFRecord và labels.tsv vào `/content/data/tct/`
3. **Reference images**: Đảm bảo có thư mục `images/` với ảnh reference

### 🔧 Troubleshooting

**Lỗi "ModuleNotFoundError: tf.contrib"**:
- Đã fix bằng tf_slim, nhưng nếu vẫn gặp, check lại cell cài đặt dependencies

**Lỗi NumPy/OpenCV ABI**:
- Đã pin NumPy 1.26.4 và OpenCV headless 4.7, không nên thay đổi

**Out of memory**:
- Giảm `BATCH_SIZE` trong config xuống 1
- Dùng ResNet50 thay vì ResNet101

### 💾 Lưu model
- Model tự động lưu vào Drive: `/content/drive/MyDrive/comparison_detector_models/`
- Checkpoint mỗi 500 steps

### 📊 Monitoring
- TensorBoard: xem loss và metrics real-time
- Logs: check terminal output để debug

---

**Repo**: https://github.com/dungdinhhaha/AIDetect  
**Tác giả**: dungdinhhaha | dungdinh542004@gmail.com